# importando librerias

In [1]:
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.utils import to_categorical
from numpy import mean, std, dstack
from matplotlib import pyplot

Using TensorFlow backend.


# Cargando el dataset

In [2]:
def cargar_archivo(pathArchivo):
    dataframe = read_csv(pathArchivo, header=None, delim_whitespace=True)
    return dataframe.values

In [3]:
def agrupar_archivos(nombreArchivos, prefijo=''):
    loaded = list()
    for nombre in nombreArchivos:
        data = cargar_archivo(prefijo + nombre)
        loaded.append(data)
    loaded = dstack(loaded)
    return loaded

In [4]:
def cargar_grupo_archivos(grupo, prefijo=''):
    pathArchivo = prefijo + grupo + '/Inertial Signals/'
    nombresArchivo = list()
    nombresArchivo += ['total_acc_x_'+grupo+'.txt', 'total_acc_y_'+grupo+'.txt', 'total_acc_z_'+grupo+'.txt']
    nombresArchivo += ['body_acc_x_'+grupo+'.txt', 'body_acc_y_'+grupo+'.txt', 'body_acc_z_'+grupo+'.txt']
    nombresArchivo += ['body_gyro_x_'+grupo+'.txt', 'body_gyro_y_'+grupo+'.txt', 'body_gyro_z_'+grupo+'.txt']
    X = agrupar_archivos(nombresArchivo, pathArchivo)
    y = cargar_archivo(prefijo + grupo + '/y_'+grupo+'.txt')
    return X, y

In [5]:
def cargar_dataset(prefijo=''):
    # load all train
    trainX, trainy = cargar_grupo_archivos('train', prefijo + './HarData/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = cargar_grupo_archivos('test', prefijo + './HarData/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

# Crear el modelo

In [6]:
trainX, trainy, testX, testy = cargar_dataset()

verbose, epochs, batch_size = 0, 15, 64
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# fit network
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

# Evaluando el modelo

In [9]:
_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

In [10]:
accuracy

0.9114353656768799